<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/notebooks/05a_NgramLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos a entrenar un modelo de lenguaje ngram sobre un corpus de recetas de cocina con la librería `nltk`.

## Configuración del entorno

In [ ]:
!pip install -qU datasets spacy nltk watermark

In [ ]:
%%capture
!python -m spacy download es_core_news_sm

In [ ]:
%reload_ext watermark

In [ ]:
%watermark -vmp datasets,spacy,nltk,numpy,pandas,tqdm

## Dataset

Vamos a usar un [corpus de recetas de SomosNLP](https://huggingface.co/datasets/somosnlp/RecetasDeLaAbuela).

In [ ]:
from datasets import load_dataset

dataset = load_dataset("somosnlp/RecetasDeLaAbuela", "version_1")

In [ ]:
# vemos la estructura:
print(dataset)

In [ ]:
# Conservamos pais = "ESP":
dataset = dataset.filter(lambda x: x["Pais"] == "ESP")

In [ ]:
# vemos un ejemplo al azar:
dataset["train"][300]

In [ ]:
# A veces los textos son listas no parseadas como tales.
# En tal caso, hacemos un join de la lista.
import re

def preprocess(example):
    """
    """
    if example["Pasos"].startswith("["):
        pasos_list = eval(example["Pasos"].encode('unicode_escape'))
        example["Pasos"] = " ".join(pasos_list)
    # Eliminamos whitespace duplicado:
    example["Pasos"] = re.sub(r'\s+', ' ', example["Pasos"])
    return example

dataset = dataset.map(preprocess)

In [ ]:
dataset["train"][300]

Hacemos un partición train/test y achicamos (solo para trabajar mas rapido). Y conservamos solo el texto de las recetas.

In [ ]:
dataset = dataset.shuffle(seed=33)

In [ ]:
texts_train = dataset["train"].select(range(0, 4_000))["Pasos"]
texts_test = dataset["train"].select(range(4_000, 8_000))["Pasos"]

In [ ]:
import textwrap

print(textwrap.fill(texts_train[33], 100))

## Tokenización y "entrenamiento" del LM

* Usamos el tokenizer para español de `spacy`.
* Consideramos como parte del vocabulario todas las palabras que ocurran al menos dos veces en train. Usamos `nltk` para definir el vocab y detectar los "\<unk\>" en test.
* Hacemos padding con BOS y EOS tokens.
* Debemos tener en cuenta que `nltk` espera que cada documento sea una lista de strings.

Usamos el LM más sencillo, el MLE (Maximum Likelihood Estimator), con 4-gramas.

In [ ]:
# tokenizer con reglas de puntacion, contracciones, etc:
import spacy

tokenizer = spacy.load('es_core_news_sm')

In [ ]:
# Veamos un ejemplo:
doc = tokenizer(texts_train[0])
print(doc.text)
for i, token in enumerate(doc):
    print(token.text)
    if i > 15:
        break

In [ ]:
# tokenizamos en train y test, sin marcar los UNK todavía:
import tqdm

def tokenize(doc, ngram_order=4):
    """Tokeniza un documento y agrega BOS y EOS tokens.
    """
    tokens = [token.text for token in tokenizer(doc)]
    tokens = ["<bos>"] * (ngram_order - 1) + tokens + ["<eos>"]
    return tokens

tokenized_train = []
for doc in tqdm.tqdm(texts_train):
    tokenized_train.append(tokenize(doc))

tokenized_test = []
for doc in tqdm.tqdm(texts_test):
    tokenized_test.append(tokenize(doc))

In [ ]:
# Un poco de hacking de nltk para evitar que haga padding con la misma cantidad de tokens
# a izq y derecha
from functools import partial
from itertools import chain

from nltk.util import everygrams, pad_sequence

flatten = chain.from_iterable
pad_both_ends = partial(
    pad_sequence,
    pad_left=True,
    left_pad_symbol="<s>",
    pad_right=True,
    right_pad_symbol="</s>",
)

def padded_everygram_pipeline(order, text):
    """Modificación de https://www.nltk.org/_modules/nltk/lm/preprocessing.html
    para que no haga padding
    """
    padding_fn = partial(pad_both_ends, n=0)
    return (
        (everygrams(list(padding_fn(sent)), max_len=order) for sent in text),
        flatten(map(padding_fn, text)),
    )

train, train_flat = padded_everygram_pipeline(4, tokenized_train)

In [ ]:
# Ahora sí armamos el vocab
from nltk.lm import Vocabulary

# cutoff de freq>=2 para el vocab:
vocab = Vocabulary(train_flat, unk_cutoff=2)

In [ ]:
# vocab size:
len(vocab)

In [ ]:
# los tokens más y menos frecuentes:
print(sorted(vocab.counts, key=vocab.counts.get, reverse=True)[:5])
print(sorted(vocab.counts, key=vocab.counts.get)[:5])

In [ ]:
# los tokens ordenados alfabeticamente:
print(sorted(vocab.counts)[:5])
print(sorted(vocab.counts, reverse=True)[:5])

In [ ]:
# los tokens con frec 1 "no están en el vocab" (pero podemos consultar su frec.)
print(vocab["el"], "el" in vocab)
print(vocab["digestivo"], "digestivo" in vocab)
print(vocab[" "], " " in vocab)
print(vocab["riquelme"], "riquelme" in vocab)

In [ ]:
# ejemplos de sequencia tokenizada:
print(vocab.lookup(tokenized_train[33][:10]))
print(vocab.lookup(["un", "té", "digestivo", "."]))

In [ ]:
# instanciamos el modelo con el ngram order y el vocab
from nltk.lm import MLE

lm = MLE(4, vocabulary=vocab)

In [ ]:
%%time
lm.fit(train)

In [ ]:
print(lm.counts)

In [ ]:
# unigram counts
lm.counts['la']

In [ ]:
# bigram counts
print(lm.counts[['en']]["la"])
print(lm.counts[['la']]["en"])
print(lm.counts[['el']]["<UNK>"])

In [ ]:
# trigram y 4gram counts
print(lm.counts[["con", "la"]]["cuchara"])
print(lm.counts[["y", "con", "la"]]["cuchara"])

In [ ]:
# lo mas frecuente despues de un ngrama dado:
ngram_example = ["con", "la"]
sorted(lm.counts[ngram_example].items(), key=lambda x: x[1], reverse=True)[:10]

In [ ]:
# probabilidad de un token luego de un ngrama:
ngram_example = ["con", "la", "salsa"]
print(lm.score("rosa", ngram_example))

In [ ]:
# usamos logscore para evitar underflow:
import numpy as np

ngram_example = ["con", "la", "salsa"]
print(lm.logscore("rosa", ngram_example))
print(np.log2(lm.score("rosa", ngram_example)))

## Evaluación

Medimos perplexity en el dataset de test.

In [ ]:
example_test = tokenized_test[33]
print(example_test)
print(lm.vocab.lookup(example_test))

In [ ]:
from nltk.util import ngrams

def perplexity(docs, lm, ngram_order=3) -> float:
    """docs: lista de listas de tokens (con BOS y EOS)
    """
    ngrams_flat = []
    for doc in docs:
        ngrams_ = ngrams(doc, ngram_order)
        ngrams_flat.extend(list(ngrams_))
    return lm.perplexity(ngrams_flat)

In [ ]:
%%time
ppl_train = perplexity(tokenized_train, lm)
print(f"Perplexity en train: {ppl_train:.4f}")

In [ ]:
%%time
ppl_test = perplexity(tokenized_test, lm)
print(f"Perplexity en test: {ppl_test:.4f}")
# qué pasó??

Necesitamos smoothing / backoff / interpolation para computar perplexity en test!

Usamos add-k smoothing (aka Lidstone smoothing, gamma=k)

In [ ]:
from nltk.lm import Lidstone

train, train_flat = padded_everygram_pipeline(4, tokenized_train)
vocab = Vocabulary(train_flat, unk_cutoff=2)
lm_smoothed = Lidstone(order=4, vocabulary=vocab, gamma=.01)

In [ ]:
%%time
lm_smoothed.fit(train)

In [ ]:
ppl_test_smoothed = perplexity(tokenized_test, lm_smoothed)
print(f"Perplexity en test: {ppl_test_smoothed:.4f}")

## Generación de texto

Generamos texto sampleando iterativamente del LM.

In [ ]:
tokens_generados = lm_smoothed.generate(
    30, text_seed=["<bos>", "<bos>", "<bos>"], random_seed=33)
print(" ".join(tokens_generados))

In [ ]:
import textwrap

tokens_generados = lm_smoothed.generate(
    120, text_seed=["<bos>", "<bos>", "1"], random_seed=33)
receta_generada = " ".join(tokens_generados)
print(textwrap.fill(receta_generada, 100))